# Model recovery attack: `wine`

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import mra
from mra import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

xy = np.loadtxt("../datasets/wine-white.csv", delimiter=";")
np.random.shuffle(xy)
x = xy[:,0:-1]
y = xy[:,-1]-3

min_values = np.array([min(x[:,i]) for i in range(x.shape[1])])
max_values = np.array([max(x[:,i]) for i in range(x.shape[1])])
x = (x-min_values)/(max_values - min_values)
y = y.reshape((len(x), 1)).astype("float32")
train_ds = make_dataset(x, y, lambda t: t)

2022-02-17 15:31:45.515975: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-17 15:31:45.516042: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: xgpg0
2022-02-17 15:31:45.516053: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: xgpg0
2022-02-17 15:31:45.516186: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 495.29.5
2022-02-17 15:31:45.516221: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 495.29.5
2022-02-17 15:31:45.516229: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 495.29.5
2022-02-17 15:31:45.516756: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perfor

In [3]:
mra_wine_3 = mra(train_ds)

In [6]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    output = tf.keras.layers.Dense(64, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    # x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(7, activation="softmax")(x)
    return tf.keras.Model(xin, output)

In [7]:
log_train = mra_wine_3.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.sparse_categorical_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=50
    )

Iteration 0001: Training loss: 1.9176
Iteration 0051: Training loss: 1.5013
Iteration 0101: Training loss: 1.1087
Iteration 0151: Training loss: 0.9151
Iteration 0201: Training loss: 1.3616
Iteration 0251: Training loss: 1.2380
Iteration 0301: Training loss: 1.1850
Iteration 0351: Training loss: 0.9829
Iteration 0401: Training loss: 1.4192
Iteration 0451: Training loss: 1.0262
Iteration 0501: Training loss: 1.3588
Iteration 0551: Training loss: 0.8540
Iteration 0601: Training loss: 1.0160
Iteration 0651: Training loss: 1.1018
Iteration 0701: Training loss: 0.9675
Iteration 0751: Training loss: 1.1136


In [8]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(512, activation=act)(xin)
    x = tf.keras.layers.Dense(256, activation=act)(x)
    x = tf.keras.layers.Dense(128, activation=act)(x)
    x = tf.keras.layers.Dense(64, activation=act)(x)
    x = tf.keras.layers.Dense(32, activation=act)(x)
    x = tf.keras.layers.Dense(11, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [10]:
log_attack = mra_wine_3.attack(
    attack_iter=100,
    make_generator=make_generator,
    input_noise="normal",
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.1735, RG-normal: 0.1621, reconstruction validation: 0.0066
Iteration 0002: RG-uniform: 0.1685, RG-normal: 0.1605, reconstruction validation: 0.0016
Iteration 0003: RG-uniform: 0.1807, RG-normal: 0.1516, reconstruction validation: 0.0008
Iteration 0004: RG-uniform: 0.1795, RG-normal: 0.1577, reconstruction validation: 0.0007
Iteration 0005: RG-uniform: 0.1568, RG-normal: 0.1473, reconstruction validation: 0.0008
Iteration 0006: RG-uniform: 0.1762, RG-normal: 0.1592, reconstruction validation: 0.0007
Iteration 0007: RG-uniform: 0.1738, RG-normal: 0.1501, reconstruction validation: 0.0010
Iteration 0008: RG-uniform: 0.1670, RG-normal: 0.1736, reconstruction validation: 0.0009
Iteration 0009: RG-uniform: 0.1712, RG-normal: 0.1500, reconstruction validation: 0.0030
Iteration 0010: RG-uniform: 0.1707, RG-normal: 0.1376, reconstruction validation: 0.0033
Iteration 0011: RG-uniform: 0.1832, RG-normal: 0.1495, reconstruction validation: 0.0050
Iteration 0012: RG-un

In [11]:
np.savetxt("./log/wine-white-3.csv", log_attack, delimiter=",")